In [1]:
import re
import numpy as np

In [2]:
def find_key(txt):
    _key = re.findall(r'ES-MA-[0-9]*-[0-9]*',txt)
    if _key:
        return _key
    else:
        return 'Null'

In [3]:
def find_temp(txt):
    temp_list = []
    tb = r'-?\d+(?:\.\d+)?(?:\s*°F)?'
    rx = r'{0}(?:\s?to\s*{0})?'.format(tb)
    results = re.findall(rx, txt)
    for item in results:
        if item[-1]=='F':
            temp_list.append(item.replace('to','-'))
    return temp_list

In [4]:
def find_time(txt):
    time_list = []
    tb = r'[+]*?\d+(?:\.\d+)?(?:\s*hours)?(?:\s*hour)?(?:\s*hours)?'
    rx = r'{0}(?:\s?to\s*{0})?'.format(tb)
    results = re.findall(rx, txt)
    for item in results:
        if item[-1:-3:-1]=='sr' or item[-1:-3:-1]=='ru':
            time_list.append((item.replace('to','-')).replace('hour','').replace('s',''))
    return time_list

In [23]:
def find_media_position(txt):
    media = ['harden','furnace','quenching','annealing',]
    word = []
    media_list = []
    for i,v in enumerate(txt.split(" ")):
        word.append(v)
    for p in range(len(word)):
        for q in range(len(media)):
            if media[q] in word[p]:
                media_list.append(p)
    return media_list

In [27]:
def find_min_position(txt):
    min_pos = []
    word = []
    for i,v in enumerate(txt.split(" ")):
        word.append(v)
    for i in range(len(find_temp(txt))):
        #print(find_temp(stmt)[i][0:6])
        pos = word.index(find_temp(txt)[i][0:6])
        min_pos.append(pos)
    return min_pos

In [35]:
def media(txt):
    media = []
    word = []
    for i,v in enumerate(txt.split(" ")):
        word.append(v)
    for i in range(len(find_min_position(txt))): #[10,23]
        lst = []
        for j in range(len(find_media_position(txt))): #[2,4,20]
            sub = abs(find_min_position(txt)[i]-find_media_position(txt)[j])
            lst.append(sub)
        media.append(word[find_media_position(txt)[lst.index(min(lst))]])
    return media

In [36]:
from collections import defaultdict
sdict = defaultdict(list)
keys_bucket = list()

data_list = list(zip(find_temp(txt),find_time(txt),media(txt)))
for data in data_list:
    if find_key(txt)!='Null':
        key = find_key(txt).pop(0)
    else:
        key = find_key(txt)
    detail = data
    
    keys_bucket.append(key)
    if key in keys_bucket:
        sdict[key].append(detail)
    else:
        sdict[key] = detail

print("\n", dict(sdict))


 {'ES-MA-46-5': [('1292°F - 1382°F', '8 ', 'harden'), ('1112°F - 1202°F', '8  - 9 ', 'furnace')]}


In [16]:
stmt=txt='ES-MA-46-5: After solution annealing; Age harden at a material temperature of 1292°F to 1382°F (700°C to 750°C) for 8 hours minimum, furnace cool to 1112°F to 1202°F (600°C to 650°C), and hold for 8 hour to 9 hours minimum. Air cool to ambient temperature.'
